In [ ]:
import numpy as np
import seaborn as sns
import seaborn.objects as so
import pandas as pd

sube2023 = pd.read_csv("dataSets/dat-ab-usos-2023.csv")
#ordeno las columnas del df

print(sube2023.head())

# Procesamiento de datos
**Ejercicio 1**

a) Visualizar el tipo de datos de cada columna. Transformar la columna DIA_TRANSPORTE
para que sea reconocida como una fecha.
Sugerencia: investigar la función to_datetime de pandas. Para completar el argumento
format, revisar la documentación de datetime

In [ ]:
sube2023["DIA_TRANSPORTE"] = pd.to_datetime(sube2023["DIA_TRANSPORTE"], format = '%Y-%m-%d')

sube2023 = sube2023.rename(columns = {"DIA_TRANSPORTE" : "FECHA"})
#Verificamos que se convirtio en el tipo que queremos
sube2023.head()

 Agregar tres columnas al DataFrame:

-  FECHA_DIA : debe indicar el nombre del día de la semana correspondiente a
DIA_TRANSPORTE

-  FECHA_ORDINAL : debe indicar el ordinal correspondiente a DIA_TRANSPORTE (por
ejemplo, a 2023-01-01 le corresponde 1, a 2023-01-02 le corresponde 2 y así sucesiva-
mente). Debe ser un entero (int).

- . FECHA_MES : debe indicar el mes correspondiente a DIA_TRANSPORTE
Sugerencia: investigar el método apply de DataFrame.

In [ ]:
#Tomamos solo el prefijo del dia
sube2023["FECHA_DIA"] = [fecha.day_name()[0:3] for fecha in sube2023["FECHA"]]
print(sube2023.head())


In [ ]:
sube2023["FECHA_ORDINAL"] = [fecha.day for fecha in sube2023["FECHA"]]
sube2023.head()

In [ ]:
sube2023["FECHA_MES"] = [fecha.month for fecha in sube2023["FECHA"]]
sube2023.head()

**Ejercicio 2**

Crear el DataFrame datos_amba, el cual sólo debe tener datos de AMBA y debe excluir datos
preliminares. Además, al ejecutar datos_amba.head() debe observarse el siguiente orden y
formato de columnas: 

In [ ]:
subeAmba = sube2023[(sube2023["AMBA"] == "SI") & (sube2023["DATO_PRELIMINAR"] == "NO")]
datos_amba = subeAmba[["FECHA","FECHA_DIA","FECHA_MES","FECHA_ORDINAL","JURISDICCION","LINEA","CANTIDAD", "TIPO_TRANSPORTE"]]
#Renombro de acorde a lo pedido
datos_amba = datos_amba.rename(columns = lambda s : "pasajeros" if s == "CANTIDAD"  else str.lower(s))  

datos_amba.head()


**Ejercicio 3.**

Utilizando datos_amba ,identificar

- la proporcion de la cantidad total anual de pasajeros que le corresponda a cada medio de transporte

In [ ]:
#La cantidad de pasajeros en cada medio de trasporte, divido el total de pasajeros, 
#multiplicado por 100 para tener porcentajes del 0% al 100%
PropXTransporte = (datos_amba.groupby("tipo_transporte")["pasajeros"].sum()/datos_amba["pasajeros"].sum() )*100
print(PropXTransporte)

- La tupla (mes, línea de subte) donde viajó la mayor cantidad de pasajeros

In [ ]:
#Me creo un nuevo dataset usando el de datos_amba, que contenga unicamente los datos relevantes,
#agregando la cantidad de pasajeros por linea por mes
GenteXLineaXMesDF= datos_amba[datos_amba["tipo_transporte"] == "SUBTE"].groupby(["fecha_mes","linea"])["pasajeros"].sum().reset_index()

#idxmax me da el indice del maximo en la fila cantidad
filaMax = GenteXLineaXMesDF.loc[GenteXLineaXMesDF["pasajeros"].idxmax()]
MesLinea = (filaMax["fecha_mes"] , filaMax["linea"])
print(MesLinea)
 

- El día hábil con menor desvío estándar en cantidad de pasajeros




In [ ]:
#Filtro por dias Hables
amba_dias_habiles = datos_amba[(datos_amba["fecha_dia"] != ("Sat")) & (datos_amba["fecha_dia"] != ("Sun"))]
#Tomo dataframe solo con las columnas que me interesan, agregando la cantidad de pasajeros por dia
pasajeros_x_dia = amba_dias_habiles.groupby(["fecha_dia","fecha"])["pasajeros"].sum().reset_index()
#A cada uno de los dias les tomo la desviacion estandar, y devuelvo el indice del minimo
pasajeros_x_dia.groupby("fecha_dia")["pasajeros"].std().idxmin()

# 4. Analisis Exploratorio

# 5. Modelado

In [ ]:
datos_ColectivoJN = datos_amba[(datos_amba["tipo_transporte"] == "COLECTIVO") & (datos_amba["jurisdiccion"] == "NACIONAL") ]
cols = datos_ColectivoJN.linea.unique() # Los nombres de las lineas de colectivo

pasajeros_por_linea = pd.DataFrame()

for col in cols:
    datos_linea = datos_ColectivoJN[datos_ColectivoJN.linea == col][["fecha", "pasajeros"]]
                           
    datos_linea = datos_linea.set_index("fecha").rename(columns = {"pasajeros" : col })
    pasajeros_por_linea = pd.concat([pasajeros_por_linea,datos_linea], axis = 1)

display(pasajeros_por_linea.head())